# **Cat or dog CNN with Keras**

First task is to organise the dataset 

In [1]:
import os
# create directories
dataset_home = 'dataset/'
subdirs = ['single_prediction/', 'training_set/', 'test_set/']
for subdir in subdirs:
    # create label subdirectories
    labeldirs = ['dogs/', 'cats/']
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        os.makedirs(newdir, exist_ok=True)

## **Creating the CNN**

In [2]:
# Librairies
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from tensorflow.keras import regularizers

In [ ]:
#method to create the model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=(64, 64, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

#iniatalisation of model
model = define_model()

#### **Training the data**

In [ ]:
import time
%time
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)

# prepare iterators
train_it = datagen.flow_from_directory('dataset/training_set/',class_mode='binary', batch_size=32, target_size=(64, 64))
test_it = datagen.flow_from_directory('dataset/test_set/',class_mode='binary', batch_size=32, target_size=(64, 64))
model.summary()

# fit model
history = model.fit(train_it, steps_per_epoch=250, validation_data=test_it, validation_steps=63, epochs=20)
#I've specified steps instead of doing them off length to try and speed up (8000/32=250 and 2000/32=62.5)


In [ ]:
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #plt.savefig(filename + '_plot.png')
    #plt.close()
    
summarize_diagnostics(history)

## **Predicting on new images**

In [ ]:
# need to make prediction path method
from keras.preprocessing import image

def new_prediction(path):
    test_image = image.load_img(path, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    results = model.predict(test_image)
    
    if results[0][0] == 1:
        return 'dog'
    else:
        return 'cat'

#new_prediction('dataset/single_prediction/cat_or_dog_1.jpg')

In [ ]:
from IPython.display import Image
Image(filename = 'dataset/single_prediction/cat_or_dog_1.jpg', width=200)

## **Improving the model **

#hyperparamter improvement options :
L2 regularization, Learning Rate (LR) optimization, Batch Size increase, Increase model capacity 

#data redesign techniques for improvement : 
Increase image resolution (progressive resizing), Random image rotations >> Change orientation of the image, Random image shifts, Vertical and horizontal shift

#Techniques for performance improvement with model optimization:
-Fine tuning the model with subset data >> Dropping few data samples for some of the overly sampled data classes
-Class weights >> Used to train highly imbalanced (biased) database, class weights will give equal importance to all the classes during training
-Fine tuning the model with train data >> Use the model to predict on training data, retrain the model for the wrongly predicted images. 

In [ ]:
#improved model
def define_model_2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.001), padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#if wanted to generate data at other anle to test it
# create data generators
#train_datagen = ImageDataGenerator(rescale=1.0/255.0,width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
#test_datagen = ImageDataGenerator(rescale=1.0/255.0)

## Another solution - transfer learning

In [ ]:
from keras.applications import vgg16 

# define cnn model
def define_model_transfer_learning():
    # load model
    model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# define the vgg16 model
classifier_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# set hidden layers as untrainable (not compulsory)
for layer in classifier_vgg16.layers:
    layer.trainable = False

# add new layers for our classification problem
flat = Flatten()(classifier_vgg16.output)
classif = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat)
output = Dense(1, activation='sigmoid')(classif)

classifier_vgg16 = Model(inputs=classifier_vgg16.inputs, outputs=output)

classifier_vgg16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#training
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (150, 150),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier_vgg16.fit(training_set,
                     steps_per_epoch = 250, #8000/32
                     epochs = 8,
                     validation_data = test_set,
                     validation_steps = 63) #2000/32=62.5